In [1]:
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch_geometric.nn import GATConv,GCNConv
from torch.nn.functional import l1_loss
import torch.nn.functional as F
import os
import numpy as np
# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device('cuda:2')
else:
    device = torch.device('cpu')
print('Using device:', device)
torch.backends.cudnn.benchmark = True


Using device: cuda:2


In [2]:
# Define your parameters
win_size = 60 # 15,30,45,60
temporal_win = 5 # 1,3,5,15
csi_last_date=['2022-12-09','2022-11-18','2022-10-28','2022-09-30']
nas_last_date=['2022-12-08','2022-11-16','2022-10-26','2022-10-05']
dataset_name = 'hs300'

if dataset_name == 'hs300':
    input_dim = 6 #6 for hs300,5 for nas100
    stock_num =  283 #283 for hs300,98 for nas100
    select_num = 30
    feature_cols=['close','open','high','low','turnover','volume']
    last_date= csi_last_date[win_size//15-1]
    m = 1.7  # 2
else :
    input_dim = 5 #6 for hs300,5 for nas100
    stock_num =  98 #283 for hs300,98 for nas100
    select_num = 10
    feature_cols=['close','high','low','open','volume']
    last_date= nas_last_date[win_size//15-1]
    m = 0.8
horizon = 1
hidden_dim = 32
out_dim = 1 
heads = 4

In [3]:
print("data loading...")
df = pd.read_csv(f'./dataset/{dataset_name}_{horizon}.csv')

def filter_extreme_3sigma(series, n=3):
    mean = series.mean()
    std = series.std()
    max_range = mean + n * std
    min_range = mean - n * std
    return np.clip(series, min_range, max_range)

def standardize_zscore(series):
    std = series.std()
    mean = series.mean()
    return (series - mean) / std
    
def process_features(df_features, feature_cols):
    df_features_grouped = df_features.groupby('dt')
    res = []
    for dt in df_features_grouped.groups:
        df = df_features_grouped.get_group(dt)
        processed_df = process_daily_df_std(df, feature_cols)
        res.append(processed_df)
    df_features = pd.concat(res)
    df_features = df_features.dropna(subset=feature_cols)
    return df_features


def process_daily_df_std(df, feature_cols):
    df = df.copy()
    for c in feature_cols:
        df[c] = df[c].replace([np.inf, -np.inf], 0)
        df[c] = filter_extreme_3sigma(df[c])
        df[c] = standardize_zscore(df[c])
    return df

data loading...


In [4]:
print('data processing...')
standardize_df = process_features(df,feature_cols)

data processing...


In [5]:
class StockDataset(Dataset):
    def __init__(self, win_size, horizon, mode):
        self.win_size = win_size
        self.horizon = horizon
        self.mode = mode
        
        self.dataset =  standardize_df
        self.dataset['dt'] = pd.to_datetime(self.dataset['dt']) 
        self.stocks = self.dataset['kdcode'].unique()

        # Preprocess data
        self.feature_slices, self.label_slices = self.preprocess_data()

    def preprocess_data(self):
        feature_slices = []
        label_slices = []
        
        for stock in self.stocks:
            stock_data = self.dataset[self.dataset['kdcode'] == stock] 

            train_data = stock_data[stock_data['dt'] < pd.to_datetime('2022-10-31')]
            val_data = stock_data[(stock_data['dt'] < pd.to_datetime('2023-01-01')) & (stock_data['dt'] > pd.to_datetime('2022-09-21'))]
            test_data = stock_data[stock_data['dt']> pd.to_datetime(last_date)] #2022-11-18 for hs300
            if self.mode == 'train':
                data = train_data
            elif self.mode == 'test':
                data = test_data
            elif self.mode == 'val':
                data = test_data
            
            feature_data=data.loc[:, feature_cols]
            label_data = np.expand_dims(data['OT'].values, axis=1)
            feature_slices.append([feature_data[i:i+self.win_size] for i in range(len(data)-self.win_size)])
            label_slices.append([label_data[i+self.win_size] for i in range(len(data)-self.win_size)])
        self.date=data['dt']
        
        feature_slices = np.transpose(np.array(feature_slices),(1,0,2,3))
        label_slices = np.transpose(np.array(label_slices),(1,0,2))
        return feature_slices, label_slices
    
    
    def __getitem__(self, index):
        return torch.FloatTensor(self.feature_slices[index]), torch.FloatTensor(self.label_slices[index])  

    def __len__(self):
        return len(self.feature_slices)
    

def collate_fn(batch):
    feature_slices = [item[0] for item in batch]
    label_slices = [item[1] for item in batch]

    feature_slices=torch.stack(feature_slices, dim=0).permute(1,0,2,3)
    label_slices=torch.stack(label_slices, dim=0).squeeze(-1).permute(1,0)
    return feature_slices.view(feature_slices.shape[0],-1,feature_slices.shape[3]),label_slices


In [6]:
train_dataset= StockDataset(win_size,horizon ,'train')
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=False, collate_fn=collate_fn)

val_dataset= StockDataset(win_size,horizon ,'val')
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False, collate_fn=collate_fn)

test_dataset = StockDataset(win_size,horizon,'test')
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, collate_fn=collate_fn)

In [7]:
class PairNorm(nn.Module):
    def __init__(self, mode='PN', scale=1):
        assert mode in ['None', 'PN', 'PN-SI', 'PN-SCS']
        super(PairNorm, self).__init__()
        self.mode = mode
        self.scale = scale

    def forward(self, x):
        if self.mode == 'None':
            return x
        col_mean = x.mean(dim=0)
        if self.mode == 'PN':
            x = x - col_mean
            rownorm_mean = (1e-6 + x.pow(2).sum(dim=1).mean()).sqrt()
            x = self.scale * x / rownorm_mean
        if self.mode == 'PN-SI':
            x = x - col_mean
            rownorm_individual = (1e-6 + x.pow(2).sum(dim=1, keepdim=True)).sqrt()
            x = self.scale * x / rownorm_individual
        if self.mode == 'PN-SCS':
            rownorm_individual = (1e-6 + x.pow(2).sum(dim=1, keepdim=True)).sqrt()
            x = self.scale * x / rownorm_individual - col_mean
        return x

In [8]:
class Model(nn.Module):
    def __init__(self, input_dim,hidden_dim,heads,t_att_heads):
        super(Model, self).__init__()
        self.multihead_attn = nn.MultiheadAttention(win_size,t_att_heads)
        self.gru1 = nn.GRU(input_dim, hidden_dim, num_layers=1, batch_first=True)
        self.gru2 = nn.GRU(input_dim, hidden_dim, num_layers=1, batch_first=True)
        self.gru3 = nn.GRU(input_dim, hidden_dim, num_layers=1, batch_first=True)
        self.gat1 = GATConv(hidden_dim*win_size, hidden_dim, heads=heads) 
        self.gat2 = GATConv(hidden_dim*win_size, hidden_dim, heads=heads) 
        self.pn = PairNorm(mode='PN-SI')
        self.predictor = nn.Sequential(nn.Linear(in_features=hidden_dim*heads, out_features=out_dim,bias=True),
                                       nn.Sigmoid())
                                       #nn.Tanh())
                                       #nn.Softsign())
        
        self.alpha = torch.nn.Parameter(torch.FloatTensor([0.5]), requires_grad=True)
        self.decay = torch.nn.Parameter(torch.FloatTensor([0.5]), requires_grad=True)
        
    def generate_edge_index(self, adj):
        I = torch.eye(adj.size(0)).to(device)
        adj+=I
        # 得到所有大于阈值的元素坐标
        indices = torch.nonzero(adj > adj.mean()+m*adj.std())

        # 转换为边索引
        edge_index = indices.t()

        return edge_index

    def forward(self, data, hs1):
        data = data.permute(0, 2, 1)
        hs1 = hs1.permute(0, 2, 1)
        embedding1, _ = self.multihead_attn(data, data, data)
        embedding1 = embedding1.permute(0, 2, 1)

        embedding2, _ = self.multihead_attn(hs1, hs1, hs1)
        embedding2 = embedding2.permute(0, 2, 1)


        out_GRU_1, _ = self.gru1(embedding1)
        out_GRU_2, _ = self.gru2(embedding1)
        out_GRU_3, _ = self.gru3(embedding2)
        
        M1 = torch.tanh(out_GRU_1.reshape(out_GRU_1.size(0), -1))
        M2 = torch.tanh(out_GRU_2.reshape(out_GRU_2.size(0), -1))
        M3 = torch.tanh(out_GRU_3.reshape(out_GRU_3.size(0), -1))
        
        adjacency_matrix_1 = F.relu(torch.tanh(torch.matmul(M1, M2.t()) - torch.matmul(M2, M1.t())))
        adjacency_matrix_2 = F.relu(torch.tanh(torch.matmul(M1, M3.t()) - torch.matmul(M3, M1.t())))

        edge_index_1 = self.generate_edge_index(adjacency_matrix_1)
        edge_index_2 = self.generate_edge_index(adjacency_matrix_2)

        GAT_output_1 = self.gat1((M1+M2)/2, edge_index_1)
        GAT_output_2 = self.gat2((M1+M3)/2, edge_index_2)

        embedding = self.pn(GAT_output_1*self.alpha+GAT_output_2*self.decay)
        output = self.predictor(embedding)

        return output,adjacency_matrix_1,edge_index_1



In [9]:
def pairwise_ranking_loss(preds, labels, margin=0.1):
    preds = preds.view(-1)
    labels = labels.view(-1)
    
    assert preds.size() == labels.size()

    # 创建一个矩阵，其中每个元素i,j都是预测值或标签差值
    diff_preds = preds[None,:] - preds[:, None]
    diff_labels = labels[None, :] - labels[:, None]
    
    # 使用几何余弦表示一致性，当预测的差值和真实标签的差值同号时，为1，否则为0
    mask = (diff_preds * diff_labels < 0).type(torch.float32)
    
    # 对差值预测应用一个线性关系，并引入一个margin
    hinge_loss = torch.nn.functional.relu(margin - diff_preds).pow(2)
    
    # 乘以mask以获得最后的损失，只有不一致的对才会有损失值
    loss = mask * hinge_loss
    return loss.sum()


In [10]:
MSE_loss = nn.MSELoss()
alpha = 1
beta= 2e-5
def combined_loss(preds,labels):
    return alpha*MSE_loss(preds,labels)+beta*pairwise_ranking_loss(preds,labels)


In [11]:
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import spearmanr, rankdata


epochs = 80
model = Model(input_dim=input_dim,hidden_dim=hidden_dim,heads=heads, t_att_heads=win_size//temporal_win).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=2e-4)
ess= []
max_ic = 0
for epoch in range(epochs):   
    model.train()
    loss_return = 0 
    es= []
    #adjacency_matrices = []
    for i, (feature_batch, label_batch) in enumerate(train_loader):
        model.zero_grad()
        feature = feature_batch.to(device)
        if i == 0:
            hs1 = feature
        target = label_batch.to(device)
        output, a , e = model(feature,hs1)
        es.append(e.shape[1])
        hs1 = feature
        loss = combined_loss(output, target)
        loss.backward()
        loss_return+=loss.item()
        optimizer.step()
        # if epoch % 15 == 0:
        #     adjacency_matrices.append(a.detach().cpu())

    print(f'Epoch {epoch+1}/{epochs}, train loss: {loss_return/i}')
    ess.append(np.mean(es))
    if (epoch+1) > 40:
        model.eval()
        with torch.no_grad():
            val_total_loss = 0
            preds = []
            targets = []
            for j, (val_data, val_target) in enumerate(val_loader):

                val_data = val_data.to(device)
                val_target = val_target.to(device)

                if j==0:
                    history = val_data

                val_output,_,_ = model(val_data,history)
                history = val_data
                val_loss = combined_loss(val_output, val_target)
                val_total_loss += val_loss.item()
                preds.append(np.array(val_output.squeeze().tolist()))
                targets.append(np.array(val_target.squeeze().tolist()))
                del val_output, val_target
        preds = np.array(preds)
        targets = np.array(targets)
        daily_rankICs = np.array([spearmanr(rankdata(preds[i]), rankdata(targets[i])).correlation for i in range(preds.shape[0])])
        mean_rankIC = np.mean(daily_rankICs)
        std_dev_rankIC = np.std(daily_rankICs)
        RankICIR = mean_rankIC / std_dev_rankIC
        if mean_rankIC > max_ic and val_total_loss/j< 0.01:
            max_ic = mean_rankIC
            torch.save(model, f=f"models/HTGNN_{dataset_name}_{horizon}_best.pth")
        print(f'Val Loss: {val_total_loss/j},Rank IC:{mean_rankIC},Rank ICIR:{RankICIR}') 
    # if epoch % 15 == 0:
    #     total_adjacency_matrix = np.sum(adjacency_matrices, axis=0)
    #     min_val = np.min(total_adjacency_matrix)
    #     max_val = np.max(total_adjacency_matrix)
    #     np.fill_diagonal(total_adjacency_matrix, 0)
    #     minmax_normalized_adjacency_matrix = (total_adjacency_matrix - min_val) / (max_val - min_val)
    #     plt.figure(figsize=(10,10))
    #     sns.heatmap(minmax_normalized_adjacency_matrix, cmap='Blues')
    #     plt.savefig(f'train_{dataset_name}_{epoch+1}.png',dpi=600)
    #     plt.show()
print(np.mean(ess))

Epoch 1/80, train loss: 0.19772631370297836
Epoch 2/80, train loss: 0.08515562067953451
Epoch 3/80, train loss: 0.05245809786827709
Epoch 4/80, train loss: 0.09569480809282345
Epoch 5/80, train loss: 0.06300803429218584
Epoch 6/80, train loss: 0.06257990393836768
Epoch 7/80, train loss: 0.0498722273915377
Epoch 8/80, train loss: 0.04770237303739487
Epoch 9/80, train loss: 0.04090680901973574
Epoch 10/80, train loss: 0.0337907151026666
Epoch 11/80, train loss: 0.07311191306619028
Epoch 12/80, train loss: 0.08943955907917589
Epoch 13/80, train loss: 0.10157731217688495
Epoch 14/80, train loss: 0.08728860807175058
Epoch 15/80, train loss: 0.06749592812382924
Epoch 16/80, train loss: 0.07053738640803146
Epoch 17/80, train loss: 0.040108238464825856
Epoch 18/80, train loss: 0.031529136252709936
Epoch 19/80, train loss: 0.027903163918640495
Epoch 20/80, train loss: 0.04844617114788152
Epoch 21/80, train loss: 0.04679032437758899
Epoch 22/80, train loss: 0.04029803343195871
Epoch 23/80, train

In [12]:
# model.load_state_dict(torch.load(f"models/HTGNN_hs300_best.pth"), strict=False)
model = torch.load(f"models/HTGNN_{dataset_name}_{horizon}_best.pth")

In [13]:
# After training, test the model
print("model testing...")
model.eval()

with torch.no_grad():
    test_total_loss = 0
    total_mae = 0
    preds = []
    targets = []
    for i, (test_data, test_target) in enumerate(test_loader):

        test_data = test_data.to(device)
        test_target = test_target.to(device)

        if i==0:
            history = test_data

        test_output,_,_ = model(test_data,history)
        history = test_data
        test_loss = MSE_loss(test_output, test_target)

        test_total_loss += test_loss.item()
        total_mae += l1_loss(test_output, test_target).item()
        preds.append(np.array(test_output.squeeze().tolist()))
        targets.append(np.array(test_target.squeeze().tolist()))
        del test_output, test_target
    # Convert lists to numpy arrays for scipy
    preds = np.array(preds)
    targets = np.array(targets)
print(f'Test MSE: {test_total_loss/i}') 
print(f'Test MAE: {total_mae/i}') 

model testing...
Test MSE: 0.000578341720368775
Test MAE: 0.018750914387961393


In [14]:
from scipy.stats import spearmanr, rankdata

# 计算每一天的 RankIC
daily_rankICs = np.array([spearmanr(rankdata(preds[i]), rankdata(targets[i])).correlation for i in range(preds.shape[0])])

# 计算 RankICIR
mean_rankIC = np.mean(daily_rankICs)
std_dev_rankIC = np.std(daily_rankICs)
RankICIR = mean_rankIC / std_dev_rankIC

print(mean_rankIC,RankICIR)

0.044044767792277194 0.2873825763814815


In [15]:
hold = np.empty((0, select_num))  # 初始化一个空的二维数组

for i in range(targets.shape[0]):
    select = np.argsort(preds[i])[::-1][:select_num]
    hold = np.vstack((hold, test_dataset.stocks[select]))


In [16]:
# test_dateset.date数据准备
dates = test_dataset.date.iloc[win_size:]

# 假设hold是一个numpy数组，将其转换为DataFrame
hold_df = pd.DataFrame(hold)

# 创建一个新DataFrame，包含日期和hold数据
combined_df = pd.concat([dates.reset_index(drop=True), hold_df], axis=1)

# 重命名列名
cols = ['dt'] + [f'kdcode{i}' for i in range(1, select_num+1)]
combined_df.columns = cols

# 保存为CSV文件
combined_df.to_csv(f'./data/hold_HTGNNpro_{dataset_name}_{horizon}.csv', index=False)


In [17]:
# 加载数据
stock_df = pd.read_csv(f'./dataset/{dataset_name}_1.csv')
hold_df = pd.read_csv(f'./data/hold_HTGNNpro_{dataset_name}_{horizon}.csv')

output = []
# 迭代hold数据
for index, row in hold_df.iterrows():
    kdcode_columns = [f'kdcode{i}' for i in range(1, select_num+1)]
    kd_codes = row[kdcode_columns].values
    dt = row['dt']

    # 获取对应的 OT 值
    ot_values = stock_df[(stock_df['kdcode'].isin(kd_codes)) & (stock_df['dt'] == dt)]['OT']

    # 计算平均值
    daily_return = ot_values.mean()

    # 将日期和对应的收益存储起来
    output.append([dt, daily_return])

# 创建DataFrame并写入文件
output_df = pd.DataFrame(output, columns=['datetime', 'daily_return'])


In [18]:
output_df.to_csv(f'./data/return_HTGNNpro_{dataset_name}_{horizon}.csv', index=False)
#torch.save(model, f=f"models/HTGNN_{dataset_name}_{horizon}_{mean_rankIC}.pth")
